In [45]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

from DataGenerator import *
from Competition import *
from config import *

In [46]:
BN_EO = initialize_BN()
dataset = BN_EO.simulate(n_samples=50000, seed=SEED)

  0%|          | 0/8 [00:00<?, ?it/s]

In [47]:
dataset

,CGPA,SCHOOL,SES,INTERN,JOB,SAT,COLLEGE,SEX
0,2,1,2,1,1,2,1,1
1,2,2,2,1,0,1,0,1
2,2,1,0,1,0,1,0,1
3,0,1,1,0,0,2,0,1
4,0,1,2,0,0,2,0,0
...,...,...,...,...,...,...,...,...
49995,1,0,0,0,0,0,0,0
49996,1,1,2,1,1,1,1,0
49997,1,0,1,1,1,0,1,0
49998,2,1,2,1,1,1,1,1


In [48]:
C1_label = 'COLLEGE'
C1_features = ['SES', 'SEX', 'SAT', 'SCHOOL']
C1 = Competition(dataset, C1_features, C1_label, ['SEX', 'SES'], [0,0])

In [49]:
C2_label = 'INTERN'
C2_features = ['SES', 'SEX', 'SAT', 'SCHOOL','COLLEGE','CGPA']
C2 = Competition(dataset, C2_features, C2_label, ['SEX', 'SES'], [0,0])

In [50]:
C3_label = 'JOB'
C3_features = ['SES', 'SEX', 'SAT', 'SCHOOL','COLLEGE','CGPA','INTERN']
C3 = Competition(dataset, C2_features, C2_label, ['SEX', 'SES'], [0,0])

In [51]:
X_train, y_train, X_test, y_test, X_val, y_val = C1.create_train_test_val_split(SEED=44)
X_train.shape, X_test.shape, X_val.shape

((30000, 4), (10000, 4), (10000, 4))

In [52]:
C2.create_train_test_val_split(SEED=44)
C3.create_train_test_val_split(SEED=44)

(       SES  SEX  SAT  SCHOOL  COLLEGE  CGPA
 7180     1    0    2       0        1     0
 23110    1    0    1       0        1     1
 39329    1    1    0       2        1     2
 319      2    1    1       1        0     2
 7916     2    0    1       0        1     1
 ...    ...  ...  ...     ...      ...   ...
 16848    2    0    2       0        1     1
 45072    1    1    0       1        0     2
 19667    2    1    2       1        1     0
 6728     2    0    2       1        1     1
 7086     0    1    0       2        1     2
 
 [30000 rows x 6 columns],
 7180     1
 23110    1
 39329    1
 319      1
 7916     1
         ..
 16848    1
 45072    0
 19667    1
 6728     1
 7086     1
 Name: INTERN, Length: 30000, dtype: int64,
        SES  SEX  SAT  SCHOOL  COLLEGE  CGPA
 49457    1    1    2       2        1     2
 20706    1    0    2       0        1     0
 46059    2    1    1       2        0     1
 38076    2    1    2       2        1     2
 8394     2    1    2       2 

In [53]:
for i in C1.test_groups.keys():
    print(i, C1.test_groups[i].shape[0]/C1.X_test.shape[0])

SEX_SES_priv 0.1191
SEX_SES_dis 0.313
SEX_priv 0.6057
SEX_dis 0.3943
SES_priv 0.2004
SES_dis 0.7996


In [54]:
best_models = {}
#for model_name in model_specs.keys():
for model_name in ['lr']:
    print(model_name)
    search = GridSearchCV(model_specs[model_name]["base_model"], model_specs[model_name]["params"], cv=5, n_jobs=-1, verbose=True)
    model = search.fit(X_train, y_train)
    best_models[model_name] = model.best_estimator_
    
best_models

lr
Fitting 5 folds for each of 25 candidates, totalling 125 fits


{'lr': LogisticRegression(C=1, max_iter=200, random_state=111, solver='newton-cg')}

In [55]:
best_model = best_models['lr']
res_df = C1.fit_base_model(best_model)

In [59]:
res2_df = C2.fit_base_model(best_model)
__ = C2.compute_predictive_metrics()

In [60]:
res3_df = C3.fit_base_model(best_model)
__ = C3.compute_predictive_metrics()

In [63]:
C3.metrics

{'overall': {'TPR': 0.8399387325291978,
  'TNR': 0.7109064266275905,
  'PPV': 0.7605755894590847,
  'FNR': 0.1600612674708022,
  'FPR': 0.2890935733724095,
  'Accuracy': 0.7783,
  'F1': 0.7982895095987627,
  'Statistical_Parity': 1.1043461612100325},
 'SEX_SES_priv': {'TPR': 0.32051282051282054,
  'TNR': 0.9655172413793104,
  'PPV': 0.6944444444444444,
  'FNR': 0.6794871794871795,
  'FPR': 0.034482758620689655,
  'Accuracy': 0.8387909319899244,
  'F1': 0.43859649122807015,
  'Statistical_Parity': 0.4615384615384615},
 'SEX_SES_dis': {'TPR': 0.9376623376623376,
  'TNR': 0.28292682926829266,
  'PPV': 0.7864923747276689,
  'FNR': 0.06233766233766234,
  'FPR': 0.7170731707317073,
  'Accuracy': 0.7661341853035144,
  'F1': 0.8554502369668247,
  'Statistical_Parity': 1.192207792207792},
 'SEX_priv': {'TPR': 0.7575635336829367,
  'TNR': 0.8138624930128563,
  'PPV': 0.7382075471698113,
  'FNR': 0.24243646631706334,
  'FPR': 0.18613750698714365,
  'Accuracy': 0.790820538220241,
  'F1': 0.7477603

In [64]:
C2.X_train

,SES,SEX,SAT,SCHOOL,COLLEGE,CGPA
7180,1,0,2,0,1,0
23110,1,0,1,0,1,1
39329,1,1,0,2,1,2
319,2,1,1,1,0,2
7916,2,0,1,0,1,1
...,...,...,...,...,...,...
16848,2,0,2,0,1,1
45072,1,1,0,1,0,2
19667,2,1,2,1,1,0
6728,2,0,2,1,1,1


In [65]:
C2.features

['SES', 'SEX', 'SAT', 'SCHOOL', 'COLLEGE', 'CGPA']

In [66]:
C2.base_model.coef_

array([[ 0.03318481,  0.50885542, -0.036908  , -0.01974558,  2.03844306,
         1.36386247]])

In [67]:
C1.X_train

,SES,SEX,SAT,SCHOOL
7180,1,0,2,0
23110,1,0,1,0
39329,1,1,0,2
319,2,1,1,1
7916,2,0,1,0
...,...,...,...,...
16848,2,0,2,0
45072,1,1,0,1
19667,2,1,2,1
6728,2,0,2,1


In [ ]:
def sigmoid(x):
    return np.array([_sigmoid_function(value) for value in x])

def _sigmoid_function(x):
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        z = np.exp(x)
        return z / (1 + z)

In [ ]:
def mse(y_true, y_pred, sample_weights=None):
    return np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weights)

In [ ]:
def cross_entropy(y_true, y_pred):
    # binary cross entropy
    y_zero_loss = y_true * np.log(y_pred + 1e-9)
    y_one_loss = (1-y_true) * np.log(1 - y_pred + 1e-9)
    return -np.mean(y_zero_loss + y_one_loss)

In [ ]:
def C_forward_linear(X, Y):
    # SES, SEX, SAT, SCHOOL --- using values from the other regression we fit
    # 0 is priv
    beta = [0.03318481,  0.50885542, -0.036908  , -0.01974558]
    beta_y = 2.03844306
    return sigmoid(np.matmul(X,beta)+beta_y*Y)

In [ ]:
class CustomLinearModel:
    """
    Linear model: Y = XB, fit by minimizing the provided loss_function
    with custom regularization
    """
    def __init__(self, loss_function=cross_entropy, C_forward=C_forward_linear,
                 X=None, Y=None, sample_weights=None, beta_init=None, 
                 regularization=0.0015):
        self.regularization = regularization
        self.beta = None
        self.loss_function = loss_function
        self.C_forward_fuction = C_forward
        self.sample_weights = sample_weights
        self.beta_init = beta_init
        
        self.model_error__ = []
        self.forward_error__ = []
        self.X = X
        self.Y = Y
            
    def predict(self, X):
        prediction = sigmoid(np.matmul(X, self.beta))
        return(prediction)

    def model_error(self):
        error = self.loss_function(
            self.predict(self.X), self.Y
        )
        #print("model error: ", error)
        self.model_error__.append(error)
        return(error)
    
    def C2_error(self):
        error = np.linalg.norm((self.C_forward_fuction(self.X, self.predict(self.X))), ord=2)
        #print("forward error: ", error)
        self.forward_error__.append(error)
        return(error)
    
    def custom_foward_loss(self, beta):
        self.beta = beta
        return (self.model_error() + (self.regularization*self.C2_error()))
    
    def l2_regularized_loss(self, beta):
        self.beta = beta
        return(self.model_error() + \
               sum(self.regularization*np.array(self.beta)**2))
    
    def plot_train_dynamics(self, scale_factor):
        plt.figure(figsize=(10,5))
        plt.plot(range(len(self.forward_error__)), [i * scale_factor for i in self.forward_error__], label='forward_err')
        plt.plot(range(len(self.forward_error__)), self.model_error__, label='model_err')
        plt.legend()
        plt.show()
    
    def fit(self, maxiter=250):        
        # Initialize beta estimates (you may need to normalize
        # your data and choose smarter initialization values
        # depending on the shape of your loss function)
        if type(self.beta_init)==type(None):
            # set beta_init = 1 for every feature
            self.beta_init = np.array([1]*self.X.shape[1])
        else: 
            # Use provided initial values
            pass
            
        if self.beta!=None and all(self.beta_init == self.beta):
            print("Model already fit once; continuing fit with more iterations.")
            
        res = minimize(self.custom_foward_loss, self.beta_init,
                       method='BFGS', options={'maxiter': maxiter})
        self.beta = res.x
        self.beta_init = self.beta

In [ ]:
lambda_range = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 100]
#lambda_range = [0, 0.0001, 0.001, 0.005, 0.01, 0.02, 0.05, 0.075, 0.1, 0.2,1]
acc = []
for lambda_ in lambda_range:
    print(lambda_)
    c1_fair = CustomLinearModel(X=C1.X_train.values, Y=C1.y_train.values, regularization=lambda_)
    c1_fair.fit(maxiter=250)
    c1_fair_preds = np.round(c1_fair.predict(C1.X_test.values))
    acc.append(accuracy_score(C1.y_test.values, c1_fair_preds))
    c1_fair.plot_train_dynamics(0.1*lambda_)
    
print(c1_fair.beta)

In [ ]:
plt.plot(lambda_range, acc)